# Grading ideas
Create a gradesheet for ideas designed as an EDSL survey.

In [1]:
# ! pip install edsl

In [2]:
from edsl import Scenario, Agent, Survey, Model
from edsl.questions import QuestionYesNo, QuestionList

Defines the notion of a "requirement" that is worth some number of points.

In [3]:
class Requirement:
    common_header = """Consider this idea for a {{ type }} business: {{ idea }}."""

    def __init__(self, requirement, grading_instructions, question_name, points):
        self.requirement = requirement
        self.grading_instructions = grading_instructions
        self.question_name = question_name
        self.points = points

    @property
    def question_text(self):
        return (self.common_header + "\n" + self.requirement + "\n" + self.grading_instructions + "\n" 
        + "Does this idea satisfy the requirement?")

    def question(self):
        return QuestionYesNo(
            question_text = self.question_text, 
            question_name = self.question_name)

r1 = Requirement(requirement = "The idea is clear and well-written.", 
    grading_instructions = ".",
    question_name = "clear_idea",
    points = 10)

r2 = Requirement(requirement = "The idea has obvious user benefits.", 
    grading_instructions = "Describe the benefits of different user types.",
    question_name = "user_benefits",
    points = 10)

r3 = Requirement(requirement = "The idea will attract a large number of new users.", 
    grading_instructions = "Explain why.",
    question_name = "new_users",
    points = 10)

requirements = [r1, r2, r3]
survey = Survey([r.question() for r in requirements])

Creates a gradesheet for the requirements:

In [4]:
class Gradesheet:
    def __init__(self, result, requirements):
        self.result = result
        self.requirements = requirements
        self.points = 0
        self._problems = []

    def grade(self):
        for requirement in self.requirements:
            answer = self.result.sub_dicts['answer'].get(requirement.question_name)
            if answer == "Yes":
                self.points += requirement.points
            # if answer == "No":
            explain = self.result.sub_dicts['answer'].get(requirement.question_name + "_comment")
            self._problems.append((requirement.question_name, str(requirement.points), explain))    

        return self.points

    def problems(self):
        return self._problems

    def sheet(self):
        from rich.console import Console
        from rich.table import Table
        console = Console()
        table = Table(show_header=True, header_style="bold magenta")

        # Add columns to the table
        table.add_column("Requirement")
        table.add_column("Points")
        table.add_column("Issue")

        for problem in self._problems:
            table.add_row(*problem)        
            table.add_row("", "", "")  # Add an empty row as a separator between items

        console.print(table)

Generates some ideas for demonstration:

In [5]:
def get_ideas(num_ideas,type=""):
    q = QuestionList(
        question_name = "ideas",
        question_text = f"Draft a list of { num_ideas } business ideas for a { type } business.",
    )
    ideas = q.run(progress_bar=True).select("ideas").to_list()[0]
    return ideas

ideas = get_ideas(5, "startup")
ideas

Output()

['Sustainable Packaging Solutions',
 'Telehealth Services',
 'Personal Finance App',
 'AI-Powered Educational Platform',
 'Smart Home Integration Services']

Create some personas for agents that will grade the ideas, runs the survey with the questions parameterized with the ideas, and then shows the results in gradesheet tables:

In [6]:
personas = [
    "You are a business analyst.",
    "You are a venture capital investor.",
    "You are an employee at a startup."
]

agents = [Agent(traits = {"persona":p} ) for p in personas]

scenarios = [Scenario({"type":"startup", "idea":i}) for i in ideas]

results = survey.by(scenarios).by(agents).run(progress_bar=True)

for i in ideas:
    for p in personas:
        print("Idea: ", i)
        print("Persona: ", p)
        g = Gradesheet(results.filter("scenario.idea == '" + i + "' and agent.persona == '" + p + "'")[0], requirements)
        print("Total points earned: ", g.grade())
        g.sheet()

Output()

Idea:  Sustainable Packaging Solutions
Persona:  You are a business analyst.
Total points earned:  20


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea 'Sustainable Packaging Solutions' is a concept rather than a fully developed  │
│               │        │ business idea. It needs more details on the business model, target market, competitive │
│               │        │ advantage, and financial projections to satisfy the requirement of a complete business │
│               │        │ idea.                                                                                  │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Sustainable Packaging Solutions would benefit multiple user types. For consumers, it   │
│               │        │ offers the peace of mind that comes with using eco-friendly products, aligning with    │
│               │        │ their values of environmental responsibility. For businesses, it can help meet         │
│               │        │ sustainability goals, reduce carbon footprint, and potentially lower long-term costs   │
│               │        │ associated with packaging. Moreover, it can enhance brand image and appeal to a        │
│               │        │ growing market of environmentally conscious customers. For governments and             │
│               │        │ communities, it contributes to waste reduction and environmental conservation efforts. │
│               │        │ Therefore, the idea satisfies the requirement of having obvious user benefits.         │
│               │        │                                                                                        │
│ new_users     │ 10     │ Sustainable Packaging Solutions is a concept that aligns well with growing             │
│               │        │ environmental concerns and the increasing demand for eco-friendly products. It targets │
│               │        │ a market with a high level of consumer awareness and regulatory pressure for           │
│               │        │ businesses to reduce their carbon footprint. As sustainability becomes a priority for  │
│               │        │ consumers and companies alike, offering sustainable packaging solutions has the        │
│               │        │ potential to attract a large number of new users who are looking to make               │
│               │        │ environmentally responsible choices.                                                   │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘

Idea:  Sustainable Packaging Solutions
Persona:  You are a venture capital investor.
Total points earned:  30


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea of Sustainable Packaging Solutions is clear and addresses a growing need for  │
│               │        │ environmentally friendly packaging options. It has potential in the current market     │
│               │        │ where sustainability is increasingly valued.                                           │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Sustainable Packaging Solutions as a startup business would indeed have obvious user   │
│               │        │ benefits. For consumers, it would provide an eco-friendly alternative to traditional   │
│               │        │ packaging, aligning with the growing demand for environmentally responsible products.  │
│               │        │ For businesses, it could mean compliance with environmental regulations, potential     │
│               │        │ cost savings through material efficiency, and a stronger brand image that appeals to   │
│               │        │ eco-conscious customers. This idea satisfies the requirement as it offers clear        │
│               │        │ advantages for different user types, both on the consumer and business sides.          │
│               │        │                                                                                        │
│ new_users     │ 10     │ The concept of Sustainable Packaging Solutions is likely to attract a large number of  │
│               │        │ new users due to the growing global concern for the environment and the increasing     │
│               │        │ demand for eco-friendly products. Consumers and businesses are becoming more conscious │
│               │        │ of their environmental footprint and are actively seeking out sustainable alternatives │
│               │        │ to traditional packaging. This trend is reinforced by regulatory pressures and the     │
│               │        │ desire for companies to enhance their brand image by committing to sustainability.     │
│               │        │ Therefore, the idea satisfies the requirement of having the potential to attract a     │
│               │        │ large user base.                                                                       │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘

...

┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea for a startup business: Smart Home Integration Services is indeed clear and   │
│               │        │ well-written. It suggests a service-based company that would focus on integrating      │
│               │        │ various smart home technologies for customers, which is a growing market.              │
│               │        │                                                                                        │
│ user_benefits │ 10     │ The idea of Smart Home Integration Services indeed has obvious user benefits. For      │
│               │        │ homeowners, it offers convenience and efficiency by allowing them to control various   │
│               │        │ aspects of their home environment with a single device or platform. For tech           │
│               │        │ enthusiasts, it provides a playground for the latest technologies and gadgets. For the │
│               │        │ elderly or those with mobility issues, it can offer increased safety and               │
│               │        │ accessibility. For energy-conscious users, it can lead to more efficient energy use    │
│               │        │ and cost savings. Therefore, this idea satisfies the requirement of having obvious     │
│               │        │ user benefits.                                                                         │
│               │        │                                                                                        │
│ new_users     │ 10     │ Smart Home Integration Services are likely to attract a large number of new users due  │
│               │        │ to the increasing demand for smart home technology and the growing interest in home    │
│               │        │ automation for convenience, energy savings, and security enhancements. As technology   │
│               │        │ advances and becomes more accessible, more consumers are looking to integrate smart    │
│               │        │ devices into their homes. A service that simplifies this integration by offering       │
│               │        │ expert advice, installation, and support can fulfill a significant market need.        │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘

Idea:  Smart Home Integration Services
Persona:  You are a venture capital investor.
Total points earned:  30


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea for Smart Home Integration Services is clear and well-defined as a potential  │
│               │        │ startup business concept.                                                              │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Smart Home Integration Services indeed have obvious user benefits. For the average     │
│               │        │ consumer, such services can offer convenience, energy efficiency, improved home        │
│               │        │ security, and personalization of their living environment. For tech-savvy users, it    │
│               │        │ provides a platform to integrate various smart devices and manage them through a       │
│               │        │ single interface, enhancing user experience. For the elderly or individuals with       │
│               │        │ disabilities, smart home technologies can offer increased independence and safety. The │
│               │        │ business users, like property developers, can use these services to increase property  │
│               │        │ value and appeal. Therefore, the idea satisfies the requirement as it presents clear   │
│               │        │ benefits for different user types.                                                     │
│               │        │                                                                                        │
│ new_users     │ 10     │ Smart Home Integration Services are increasingly in demand as more consumers are       │
│               │        │ looking to incorporate smart technology into their homes for convenience, energy       │
│               │        │ efficiency, and security. As the Internet of Things (IoT) continues to expand, the     │
│               │        │ number of devices and the complexity of integration grow, creating a significant       │
│               │        │ market opportunity for services that can seamlessly integrate these technologies. This │
│               │        │ startup idea is likely to attract a large number of new users who are interested in    │
│               │        │ upgrading their living spaces with smart home technology but require assistance in     │
│               │        │ installation and integration.                                                          │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘

Idea:  Smart Home Integration Services
Persona:  You are an employee at a startup.
Total points earned:  30


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Requirement   ┃ Points ┃ Issue                                                                                  ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ clear_idea    │ 10     │ The idea for 'Smart Home Integration Services' is clear and indicates a service that   │
│               │        │ would help integrate various smart home devices and systems for customers.             │
│               │        │                                                                                        │
│ user_benefits │ 10     │ Smart Home Integration Services offer numerous benefits for different user types. For  │
│               │        │ instance, tech-savvy users can enjoy the convenience and customization of controlling  │
│               │        │ their home environment with their devices. Elderly or disabled individuals can benefit │
│               │        │ from the enhanced accessibility and automation that smart home technology provides.    │
│               │        │ Busy professionals might appreciate the time savings and efficiency gained from having │
│               │        │ a connected home that can manage itself in many ways, such as adjusting the thermostat │
│               │        │ or lighting based on their habits. Families can enjoy increased security and peace of  │
│               │        │ mind through smart locks and security systems. Therefore, this idea satisfies the      │
│               │        │ requirement as it has clear user benefits for a variety of user types.                 │
│               │        │                                                                                        │
│ new_users     │ 10     │ Smart Home Integration Services are increasingly in demand as more consumers are       │
│               │        │ looking to make their homes connected and automated. The convenience, energy           │
│               │        │ efficiency, and enhanced security features of smart homes appeal to a broad audience,  │
│               │        │ from tech-savvy individuals to those seeking convenience and improved quality of life. │
│               │        │ Therefore, the idea is likely to attract a large number of new users, satisfying the   │
│               │        │ requirement.                                                                           │
│               │        │                                                                                        │
└───────────────┴────────┴────────────────────────────────────────────────────────────────────────────────────────┘